




# Garbage









Oversampling with SMOTE to fix the unbalanced classes.

This creates new data similar to the original one in order to have a number
of samples which is similar (equal) for every class. In this way we introduce more generalization
capability to our classification models

In [ ]:
# SMOTE --> balance the number of entries of each class
from imblearn.over_sampling import SMOTE
smote = SMOTE(
    sampling_strategy='not majority',    # resample all classes but the majority one
    k_neighbors=5
)

train_set, train_labels = smote.fit_resample(train_set, train_labels)
test_set, test_labels = smote.fit_resample(test_set, test_labels)

In [ ]:
assert(len(train_labels) == len(train_labels))
assert(len(test_set) == len(test_labels))
print(f"{len(train_labels)} training samples")
print(f"{len(test_labels)} test samples")

In [ ]:
train_set.boxplot()

Rescale the datasets: it's better when using neural networks

In [ ]:
from sklearn.preprocessing import MinMaxScaler
cols = train_set.columns
scaler = MinMaxScaler()
train_set = scaler.fit_transform(train_set.values)
train_set = pd.DataFrame(train_set, columns=cols)
test_set = scaler.fit_transform(test_set.values)
test_set = pd.DataFrame(test_set, columns=cols)

In [ ]:
train_set.boxplot()

In [ ]:
# Encode labels with one-hot
train_labels = to_categorical(train_labels, 3)
test_labels = to_categorical(test_labels, 3)

Create the Neural Network

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(1,13)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Fit the model

In [ ]:
history = model.fit(
    train_set,
    train_labels,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Plot training results

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'b', label='Training Acc')
plt.plot(epochs, val_acc, 'bo', label='Validation Acc')
plt.title('Training and validation Acc')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()

Make predictions on the test set

In [ ]:
test_predictions = model.predict_classes(test_set)
test_predictions = to_categorical(test_predictions, 3)

In [ ]:
# Function to create reports on the classification results
from sklearn.metrics import classification_report
classes = ['High_Spend','Low_Spend','Med_Spend']
def report_scores(test_label, test_pred):
    print(
        classification_report(
            test_label,
            test_pred,
            target_names=classes
        )
    )

In [ ]:
report_scores(test_labels, test_predictions)

In [ ]:
#test_loss, test_accuracy = model.evaluate(test_set, test_label, batch_size=32, verbose=1)
#print(f"\nTest accuracy: {test_accuracy}")

